# Fine-tune ruBERT-tiny2

Загружаем библиотеки

In [19]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
import nltk
from transformers import AutoModel, AutoTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric, Dataset
from sklearn.metrics import classification_report, f1_score

Загружаем отобранные статьи 

In [20]:
with open('datanews.json', encoding="utf-8") as f:
    read_data = f.read()
read_data = read_data.replace('\n][\n', ',\n')
articles = pd.read_json(read_data, orient='records')

Формируем предложения из статей. Второе предложение типа "МОСКВА, 12 янв – РИА Новости" убираем.
В новый DataFrame записываем предложени и их длину

In [21]:
pst = nltk.PunktSentenceTokenizer()
sentences = pd.DataFrame({"sentence": [], "len": []})
for ind in articles.index:
    sentArticle = pst.tokenize(articles['Article'][ind])
    m = 0
    for s in sentArticle:
        if m != 1:
            sentences.loc[len(sentences.index)] = [s, len(s)]
print("Number of sentences: ", len(sentences.index))
sentences.head(3)

Number of sentences:  222


,sentence,len
0,В Госдуме предложили сделать старый Новый год ...,84
1,Председатель союза дачников Подмосковья и депу...,234
2,"""После длинных новогодних праздников людям тяж...",176


In [22]:
train_text = sentences['sentence'].astype('str')

Задание всех seed

In [23]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(42)

In [24]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /cointegrated/rubert-tiny2/resolve/main/tokenizer_config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:997)')))"), '(Request ID: 3dcd38a5-635f-4f99-9468-df4c9bb7957f)')